<a href="https://colab.research.google.com/github/paul-kim88/pythonProject2/blob/master/scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lxml[html_clean] readability-lxml pdfminer.six

In [16]:
import requests
from bs4 import BeautifulSoup
from readability import Document
import pandas as pd
import re, html, time
from io import BytesIO
from pdfminer.high_level import extract_text

def duckduckgo_scrape(query, time='d'):
    url = f"https://duckduckgo.com/html/?q={query}&t=h_&df={time}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        results = []
        for result in soup.find_all("div", class_="result"):
            title_element = result.find("a", class_="result__a")
            if title_element:
                title = title_element.text
                link = title_element["href"]
                results.append({"title": title, "link": link})
        return results
    except Exception as e:
        print(f"Error during request: {e}")
        return None

def extract(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    text = soup.get_text(separator="\n", strip=True)
    return html.unescape(text)

def process_link(result, session):
    try:
        response = session.get(result['link'], timeout=10)
        response.raise_for_status()

        if result['link'].endswith('.pdf') or 'application/pdf' in response.headers.get('Content-Type', ''):
            pdf_file = BytesIO(response.content)
            content = extract_text(pdf_file)
        else:
            doc = Document(response.content)
            content = extract(doc.summary())
        if len(content) < 100:
            return None
        return {"title": result['title'], "link": result['link'], "content": content}

    except Exception as e:
        print(f"Error processing {result['link']}: {e}")
        return None

if __name__ == "__main__":
    search_query = input("Type What you want : ")
    start_time = time.time()
    search_results = duckduckgo_scrape(search_query, '3d')
    results = []

    if search_results:
        with requests.Session() as session:
            for result in search_results:
                if 'wiki' in result['link']:
                    continue
                # print(f"Title: {result['title']}")
                # print(f"Link: {result['link']}")
                # print("-" * 20)
                processed = process_link(result, session)
                if processed:
                    results.append(processed)

    search_df = pd.DataFrame(results)
    end_time = time.time()
    print("실행 시간:", end_time - start_time, "초")


Type What you want : LG화학 친환경소재
실행 시간: 18.439353704452515 초


In [17]:
search_df

,title,link,content
0,세계 최초 생분해성 신소재를 개발한 Lg화학! 친환경 소재를 개발해 자원 선순환에 ...,https://blog.lgchem.com/2020/11/16_biodegradab...,LG화학은 지속가능한 경영을 위해 환경을 생각하며 자원 선순환에 앞장서고 있습니다....
1,친환경 바이오 밸런스드(Bio-balanced) 제품으로 국내 최초 친환경 국제인증...,https://blog.lgchem.com/2021/07/12_bio_balanced/,최근 들어 ‘지속가능성’이란 단어를 한 번쯤은 들어보셨을 텐데요. 지속가능성이란 정...
2,화장품 등 일상에서도 친환경 소재가 쓰이도록! 친환경 소재를 개발하고 재활용 순한 ...,https://blog.lgchem.com/2023/11/09_eco_friendl...,환경에 대한 관심이 높아지면서 여러 기업에서도 재활용 플라스틱을 활용하는 범위가 점...
3,PDF Product Book - LG Chem,https://www.lgchem.com/upload/file/sustainabil...,LETZero \nProduct Book \n\nRECYCLED\n\nCOMPOST...
4,Lg화학 전구체 프리 양극재 국내 최초 양산 │ 매거진한경,https://magazine.hankyung.com/business/article...,LG화학 인터배터리2025 부스 조감도. 사진=LG화학\nLG화학이 국내 최초로 전...
5,Lg화학 친환경 소재 등에 2025년까지 10조 투자 | 한국경제,https://www.hankyung.com/article/2021090818801,입력\n2021.09.08 17:58\n수정\n2021.09.09 01:15\n지면...
6,"Lg화학, 재활용 플라스틱 더한 친환경 난연 소재 개발 | 연합뉴스",https://www.yna.co.kr/view/AKR20240812021000003,美 인증 표준 난연 테스트서 PC/ABS 소재로 첫 V-O 등급\n(서울=연합뉴스)...


In [18]:
import requests
from bs4 import BeautifulSoup
from readability import Document
import pandas as pd
import re, html
from io import BytesIO
from pdfminer.high_level import extract_text
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

def duckduckgo_scrape(query, time='d'):
    url = f"https://duckduckgo.com/html/?q={query}&t=h_&df={time}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        results = []
        for result in soup.find_all("div", class_="result"):
            title_element = result.find("a", class_="result__a")
            if title_element:
                title = title_element.text
                link = title_element["href"]
                results.append({"title": title, "link": link})
        return results
    except Exception as e:
        print(f"Error during request: {e}")
        return None

def extract(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    text = soup.get_text(separator="\n", strip=True)
    return html.unescape(text)

def process_link(result, session):
    try:
        response = session.get(result['link'], timeout=10)
        response.raise_for_status()

        if result['link'].endswith('.pdf') or 'application/pdf' in response.headers.get('Content-Type', ''):
            pdf_file = BytesIO(response.content)
            content = extract_text(pdf_file)
        else:
            doc = Document(response.content)
            content = extract(doc.summary())
        if len(content) < 100:
            return None  # 내용이 100자 미만이면 건너뜁니다.
        return {"title": result['title'], "link": result['link'], "content": content}
    except Exception as e:
        print(f"Error processing {result['link']}: {e}")
        return None

if __name__ == "__main__":
    search_query = input("Type What you want : ")
    start_time = time.time()
    search_results = duckduckgo_scrape(search_query, '3d')
    results = []

    if search_results:
        with requests.Session() as session:
          with ThreadPoolExecutor(max_workers=20) as executor:
              future_to_result = {}
              for result in search_results:
                  if 'wiki' in result['link']:
                      continue
                  # print(f"Title: {result['title']}")
                  # print(f"Link: {result['link']}")
                  # print("-" * 20)

                  future = executor.submit(process_link, result, session)
                  future_to_result[future] = result

              for future in as_completed(future_to_result):
                  processed = future.result()
                  if processed:
                      results.append(processed)

    search_df = pd.DataFrame(results)
    end_time = time.time()
    print("실행 시간:", end_time - start_time, "초")


Type What you want : LG화학 친환경소재
실행 시간: 4.114607334136963 초


In [19]:
search_df

,title,link,content
0,Lg화학 친환경 소재 등에 2025년까지 10조 투자 | 한국경제,https://www.hankyung.com/article/2021090818801,입력\n2021.09.08 17:58\n수정\n2021.09.09 01:15\n지면...
1,"Lg화학, 재활용 플라스틱 더한 친환경 난연 소재 개발 | 연합뉴스",https://www.yna.co.kr/view/AKR20240812021000003,美 인증 표준 난연 테스트서 PC/ABS 소재로 첫 V-O 등급\n(서울=연합뉴스)...
2,①불에 잘 안 타면서 ②재활용 플라스틱까지 더한 친환경 난연 소재 개발한 Lg화학 ...,https://www.hankookilbo.com/News/Read/A2024081...,PFAS 없이도 불에 잘 안타면서 환경 친화적\n미국 난연 테스트에서 첫 V-0 등...
3,세계 최초 생분해성 신소재를 개발한 Lg화학! 친환경 소재를 개발해 자원 선순환에 ...,https://blog.lgchem.com/2020/11/16_biodegradab...,LG화학은 지속가능한 경영을 위해 환경을 생각하며 자원 선순환에 앞장서고 있습니다....
4,생분해성 플라스틱의 핵심 원료 3hp! Lg화학이 세계 최초로 상용화에 도전합니다,https://blog.lgchem.com/2023/11/16_3hp/,기후 위기 앞에 인류는 지구 보호를 위해 탄소 배출 규제를 점점 강화하고 플라스틱 ...
5,친환경 바이오 밸런스드(Bio-balanced) 제품으로 국내 최초 친환경 국제인증...,https://blog.lgchem.com/2021/07/12_bio_balanced/,최근 들어 ‘지속가능성’이란 단어를 한 번쯤은 들어보셨을 텐데요. 지속가능성이란 정...
